# Ensemble methods. Exercises


In this section we have only two exercise:

1. Find the best three classifier in the stacking method using the classifiers from scikit-learn package.

2. Build arcing arc-x4 method. 

In [111]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

## Exercise 1: Find the best three classifier in the stacking method

Please use the following classifiers:

* Linear regression,
* Nearest Neighbors,
* Linear SVM,
* Decision Tree,
* Naive Bayes,
* QDA.

In [112]:
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [113]:
def build_classifiers():

    classifiers = [LinearRegression(), KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), GaussianNB(), QuadraticDiscriminantAnalysis()]
    classify = []
    
    for classifier in classifiers:
        classifier.fit(data_set, labels)
        classify.append(classifier)
    
    return classify

In [114]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier = DecisionTreeClassifier()
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

In [115]:
import itertools as it

In [116]:
classifiers = build_classifiers()
permutations = it.permutations(classifiers, 3)

classifier = None
best_accuracy = 0

for permutation in permutations:
    predicted = build_stacked_classifier(permutation)
    accuracy = accuracy_score(test_labels, predicted)
    print(f"Classifier for methods: {classifier} with accuracy {accuracy}")

    if (accuracy > best_accuracy):
        classifier = permutation
        best_accuracy = accuracy
print("\n\n")
print(f"Best ensembled classifier is for methods: {classifier} with accuracy {best_accuracy}")

Classifier for methods: None with accuracy 0.1
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), SVC()) with accuracy 0.7
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), DecisionTreeClassifier()) with accuracy 0.7
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), DecisionTreeClassifier()) with accuracy 0.75
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()) with accuracy 0.65
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()) with accuracy 0.6
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()) with accuracy 0.6
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()) with accuracy 0.6
Classifier for methods: (LinearRegression(), KNeighborsClassifier(), QuadraticDiscriminantAnalysis()) with accuracy 0.05
Classifier for methods: (Linea

## Exercise 2: 

Use the boosting method and change the code to fullfilt the following requirements:

* the weights should be calculated as:
$w_{n}^{(t+1)}=\frac{1+ I(y_{n}\neq h_{t}(x_{n})}{\sum_{i=1}^{N}1+I(y_{n}\neq h_{t}(x_{n})}$,
* the prediction is done with a voting method.

In [117]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

# prepare data set

def generate_data(sample_number, feature_number, label_number):
    data_set = np.random.random_sample((sample_number, feature_number))
    labels = np.random.choice(label_number, sample_number)
    return data_set, labels

labels = 2
dimension = 2
test_set_size = 1000
train_set_size = 5000
train_set, train_labels = generate_data(train_set_size, dimension, labels)
test_set, test_labels = generate_data(test_set_size, dimension, labels)

# init weights
number_of_iterations = 10
weights = np.ones((test_set_size,)) / test_set_size


def train_model(classifier, weights):
    return classifier.fit(X=test_set, y=test_labels, sample_weight=weights)

def calculate_error(model):
    predicted = model.predict(test_set)
    I=calculate_accuracy_vector(predicted, test_labels)
    Z=np.sum(I)
    return (1+Z)/1.0

In [118]:
def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(0)
        else:
            result.append(1)
    return result

Fill the two functions below:

In [119]:
def set_new_weights(model):
    predicted = model.predict(test_set)
    I = np.array(calculate_accuracy_vector(predicted, test_labels))
    suma = np.sum(I)
    normalize = (I + 1)/ suma
    
    return normalize


Train the classifier with the code below:

In [120]:
classifier = DecisionTreeClassifier(max_depth=1, random_state=1)
classifier.fit(X=train_set, y=train_labels)
alphas = []
classifiers = []
for iteration in range(number_of_iterations):
    model = train_model(classifier, weights)
    weights = set_new_weights(model)
    classifiers.append(model)

print(weights)


validate_x, validate_label = generate_data(1, dimension, labels)

[0.00379507 0.00189753 0.00189753 0.00189753 0.00379507 0.00379507
 0.00379507 0.00189753 0.00189753 0.00189753 0.00189753 0.00379507
 0.00379507 0.00379507 0.00379507 0.00189753 0.00379507 0.00189753
 0.00189753 0.00379507 0.00379507 0.00189753 0.00189753 0.00379507
 0.00379507 0.00379507 0.00379507 0.00189753 0.00189753 0.00379507
 0.00189753 0.00379507 0.00189753 0.00379507 0.00189753 0.00379507
 0.00189753 0.00189753 0.00189753 0.00379507 0.00379507 0.00379507
 0.00189753 0.00189753 0.00379507 0.00189753 0.00379507 0.00379507
 0.00189753 0.00189753 0.00379507 0.00379507 0.00379507 0.00379507
 0.00379507 0.00379507 0.00379507 0.00379507 0.00379507 0.00189753
 0.00189753 0.00379507 0.00189753 0.00379507 0.00189753 0.00379507
 0.00189753 0.00189753 0.00379507 0.00189753 0.00379507 0.00379507
 0.00379507 0.00189753 0.00189753 0.00189753 0.00189753 0.00189753
 0.00379507 0.00189753 0.00189753 0.00189753 0.00379507 0.00189753
 0.00189753 0.00379507 0.00189753 0.00379507 0.00189753 0.0037

Set the validation data set:

In [121]:
validate_x, validate_label = generate_data(1, dimension, labels)

Fill the prediction code:

In [122]:
def get_prediction(x):
    dictionary = {
        0 : 0,
        1 : 0
    }
    predictions = []
    for classifier in classifiers:
        predicted = classifier.predict(x)
        predictions.append(predicted)

    for i, selected_class in enumerate(predictions):
        print(selected_class)
        dictionary[selected_class.item()] += weights[selected_class]

    if dictionary[0] >= dictionary[1]:
        return 0
    else:
        return 1

Test it:

In [123]:
prediction = get_prediction(validate_x)


print(f"More probable it is class '{prediction}'")

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
More probable it is class '1'
